In [1]:
import numpy as np
import pandas as pd
import logging
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt
import magic
import time

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from math import *

In [2]:
def build_and_compile_model(optim,lr):
    model = keras.Sequential([
        layers.Dense(250, activation='relu'), # sequential NN with descending layers sizes
        layers.Dense(200, activation='relu'),
        layers.Dense(150, activation='relu'),
        layers.Dense(134)
    ])
    model.compile(loss=tf.keras.metrics.mean_squared_error,
                  metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')], # using RMSE b/c for values <1 MSE may
                  optimizer=optim(lr))                     # make the error look smaller than it actually is
    return model
def plot_loss(history):
    '''
    plots the loss function for the training and validation data
    '''
    plt.plot(np.array(history.history['loss'])**0.5, label='loss')
    plt.plot(np.array(history.history['val_loss'])**0.5, label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('RMSE')
    plt.legend()
    plt.grid(True)

In [3]:
train_gex = ad.read_h5ad("Gex_processed_training.h5ad") # gex is gene expression which are RNA; Training data input
train_adt = ad.read_h5ad("Adt_processed_training.h5ad") # adt is protein; Training data response

test_gex = ad.read_h5ad("Gex_processed_testing.h5ad") # gex is gene expression which are RNA; Training data input
test_adt = ad.read_h5ad("Adt_processed_testing.h5ad") # adt is protein; Training data response

data = ad.concat([train_gex,test_gex])
targets = ad.concat([train_adt,test_adt])

In [4]:
train_gex.obsm['Normalized'] = normalize(train_gex.X.toarray())
test_gex.obsm['Normalized'] = normalize(test_gex.X.toarray())

In [5]:
magic_operator = magic.MAGIC()
X_magic_train = magic_operator.fit_transform(train_gex.obsm['Normalized'])

Calculating MAGIC...
  Running MAGIC on 42123 cells and 13953 genes.
  Calculating graph and diffusion operator...
    Calculating PCA...
    Calculated PCA in 23.76 seconds.
    Calculating KNN search...
    Calculated KNN search in 89.39 seconds.
    Calculating affinities...
    Calculated affinities in 87.22 seconds.
  Calculated graph and diffusion operator in 200.42 seconds.
  Running MAGIC with `solver='exact'` on 13953-dimensional data may take a long time. Consider denoising specific genes with `genes=<list-like>` or using `solver='approximate'`.
  Calculating imputation...
  Calculated imputation in 29.05 seconds.
Calculated MAGIC in 230.08 seconds.


In [6]:
magic_operator = magic.MAGIC()
X_magic_test = magic_operator.fit_transform(test_gex.obsm['Normalized'])

Calculating MAGIC...
  Running MAGIC on 24052 cells and 13953 genes.
  Calculating graph and diffusion operator...
    Calculating PCA...
    Calculated PCA in 12.31 seconds.
    Calculating KNN search...
    Calculated KNN search in 20.82 seconds.
    Calculating affinities...
    Calculated affinities in 19.74 seconds.
  Calculated graph and diffusion operator in 52.90 seconds.
  Running MAGIC with `solver='exact'` on 13953-dimensional data may take a long time. Consider denoising specific genes with `genes=<list-like>` or using `solver='approximate'`.
  Calculating imputation...
  Calculated imputation in 16.26 seconds.
Calculated MAGIC in 70.66 seconds.


In [7]:
del train_gex
del test_gex

In [18]:
pca_trans = PCA(n_components=50)

In [19]:
pca_res = pca_trans.fit_transform(X_magic_train)

In [20]:
old_comp = pca_trans.components_

In [21]:
train_mean = np.mean(X_magic_train)

In [22]:
X_test_meaned = X_magic_test-train_mean
X_test_pca = np.dot(X_test_meaned,old_comp.transpose())

In [23]:
MLP_model = build_and_compile_model(tf.keras.optimizers.Adamax,0.001)
history = MLP_model.fit(pca_res,train_adt.X.toarray(),validation_split=0.15,epochs=40,batch_size=16)

Epoch 1/40
2238/2238 [==============================] - 4s 2ms/step - loss: 0.1727 - rmse: 0.4155 - val_loss: 0.1083 - val_rmse: 0.3290
Epoch 2/40
2238/2238 [==============================] - 4s 2ms/step - loss: 0.1171 - rmse: 0.3421 - val_loss: 0.0986 - val_rmse: 0.3141
Epoch 3/40
2238/2238 [==============================] - 4s 2ms/step - loss: 0.1107 - rmse: 0.3328 - val_loss: 0.0964 - val_rmse: 0.3105
Epoch 4/40
2238/2238 [==============================] - 4s 2ms/step - loss: 0.1076 - rmse: 0.3280 - val_loss: 0.0963 - val_rmse: 0.3103
Epoch 5/40
2238/2238 [==============================] - 4s 2ms/step - loss: 0.1055 - rmse: 0.3248 - val_loss: 0.0955 - val_rmse: 0.3090
Epoch 6/40
2238/2238 [==============================] - 4s 2ms/step - loss: 0.1041 - rmse: 0.3227 - val_loss: 0.0940 - val_rmse: 0.3066
Epoch 7/40
2238/2238 [==============================] - 4s 2ms/step - loss: 0.1031 - rmse: 0.3211 - val_loss: 0.0922 - val_rmse: 0.3036
Epoch 8/40
2238/2238 [==========================

In [41]:
pred = MLP_model.predict(X_test_pca)

752/752 [==============================] - 1s 789us/step


In [42]:
mean_squared_error(test_adt.X.toarray(),pred,squared=False)

0.9983102312650861

In [33]:
data = np.vstack([X_magic_train,X_magic_test])

In [49]:
pca = PCA(n_components=50)
pca_ex = pca.fit_transform(data)

In [50]:
X_train_ex = pca_ex[:len(X_magic_train)]
X_test_ex = pca_ex[len(X_magic_train):]

In [37]:
X_train_ex.shape

(42123, 50)

In [51]:
X_test_ex.shape

(24052, 50)

In [52]:
MLP_model = build_and_compile_model(tf.keras.optimizers.Adamax,0.001)
history = MLP_model.fit(X_train_ex,train_adt.X.toarray(),validation_split=0.15,epochs=40,batch_size=16)

Epoch 1/40
2238/2238 [==============================] - 5s 2ms/step - loss: 0.1715 - rmse: 0.4141 - val_loss: 0.1064 - val_rmse: 0.3262
Epoch 2/40
2238/2238 [==============================] - 4s 2ms/step - loss: 0.1174 - rmse: 0.3426 - val_loss: 0.1013 - val_rmse: 0.3183
Epoch 3/40
2238/2238 [==============================] - 4s 2ms/step - loss: 0.1108 - rmse: 0.3329 - val_loss: 0.0979 - val_rmse: 0.3129
Epoch 4/40
2238/2238 [==============================] - 4s 2ms/step - loss: 0.1075 - rmse: 0.3279 - val_loss: 0.0967 - val_rmse: 0.3110
Epoch 5/40
2238/2238 [==============================] - 4s 2ms/step - loss: 0.1054 - rmse: 0.3247 - val_loss: 0.0936 - val_rmse: 0.3059
Epoch 6/40
2238/2238 [==============================] - 4s 2ms/step - loss: 0.1040 - rmse: 0.3225 - val_loss: 0.0942 - val_rmse: 0.3069
Epoch 7/40
2238/2238 [==============================] - 4s 2ms/step - loss: 0.1028 - rmse: 0.3207 - val_loss: 0.0928 - val_rmse: 0.3046
Epoch 8/40
2238/2238 [==========================

In [53]:
pred = MLP_model.predict(X_test_ex)
mean_squared_error(test_adt.X.toarray(),pred,squared=False)

752/752 [==============================] - 1s 649us/step


0.35640544129138524